In [160]:
import sys
!{sys.executable} -m pip install transformers
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install sentencepiece
!{sys.executable} -m pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

## Tokenizers


In [161]:
from transformers import AutoModel, AutoTokenizer
m_path = "t5-small"#"bert-base-uncased"
# model = AutoModel.from_pretrained(m_path)
tokenizer_t5 = AutoTokenizer.from_pretrained("t5-small")
tokenizer_bert = AutoTokenizer.from_pretrained("bert-base-uncased")


In [174]:
input_sentence = "summarize: This is an input textwhich is blablabla. \n [SEP] "
input_tokenized = tokenizer_bert.tokenize(input_sentence)
print(input_tokenized)
input_tokenized2 = tokenizer_bert.encode(input_sentence)
print(input_tokenized2)
text_token=tokenizer_bert.convert_ids_to_tokens(input_tokenized2)
print(text_token)
print(tokenizer_bert(input_sentence))

['sum', '##mar', '##ize', ':', 'this', 'is', 'an', 'input', 'text', '##w', '##hic', '##h', 'is', 'b', '##lab', '##lab', '##la', '.', '[SEP]']
[101, 7680, 7849, 4697, 1024, 2023, 2003, 2019, 7953, 3793, 2860, 16066, 2232, 2003, 1038, 20470, 20470, 2721, 1012, 102, 102]
['[CLS]', 'sum', '##mar', '##ize', ':', 'this', 'is', 'an', 'input', 'text', '##w', '##hic', '##h', 'is', 'b', '##lab', '##lab', '##la', '.', '[SEP]', '[SEP]']
{'input_ids': [101, 7680, 7849, 4697, 1024, 2023, 2003, 2019, 7953, 3793, 2860, 16066, 2232, 2003, 1038, 20470, 20470, 2721, 1012, 102, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


## Model Architecture

In [163]:
input_sentence = "This is the supercalifragilisticexpialidocious input text."
tokenized_input = tokenizer_t5(input_sentence)

print("Original output: {}".format(input_sentence))
print("Tokenized output (in IDs): {}".format(tokenized_input.input_ids))
print("Tokenized output (in tokens): {}".format(tokenizer_t5.convert_ids_to_tokens(tokenized_input.input_ids)))

Original output: This is the supercalifragilisticexpialidocious input text.
Tokenized output (in IDs): [100, 19, 8, 1355, 15534, 20791, 173, 3040, 994, 102, 23, 4288, 7171, 2936, 3785, 1499, 5, 1]
Tokenized output (in tokens): ['▁This', '▁is', '▁the', '▁super', 'cali', 'frag', 'il', 'istic', 'ex', 'p', 'i', 'ali', 'doc', 'ious', '▁input', '▁text', '.', '</s>']


## Embeddings
First we need to transform our tokenized input into a vector representation. This representantion needs to hold information about the semantic property of a word (*Input Embedding*) and a positional information about where the word is in the input sequence (*Positional Embedding*). Each word's embedding has a size of a hidden layer (default 512). 


In [164]:
model = T5ForConditionalGeneration.from_pretrained(m_path)
input_ids = tokenizer_t5(input_sentence, return_tensors="pt")
outputs=model.generate(input_ids.input_ids, return_dict_in_generate=True,output_hidden_states=True, output_attentions=True, output_scores=True)

word_embeddings_for_input_sentence = outputs.encoder_hidden_states[0] 
print("Size of the input ids tensor: {}".format(input_ids.input_ids.shape))
print("Size of the word embedding for the input sequence: {}".format(word_embeddings_for_input_sentence.shape))

Size of the input ids tensor: torch.Size([1, 18])
Size of the word embedding for the input sequence: torch.Size([1, 18, 512])



## What does Self-attention do?
Self-attention layer is a building block of the Transformer infrastructure, which is able to include the information about the connection between  individual words/tokens from the input sequence. 

The image bellow shows from which the computation of self-attention consists. Each of the listed matrices (Q, K and V) is computed by multiplication of the matrix of input embeddings and a weight matrices ($W^{Q}$, $W^{K}$, $W^{V}$). These matrices contain the parameters which are trained during training. 

![image.png](https://jalammar.github.io/images/t/self-attention-matrix-calculation-2.png)
[[source]](https://jalammar.github.io/illustrated-transformer/)

Each self-attention head has its own weight matrices ($W^{Q}$, $W^{K}$, $W^{V}$), which are trained independently of each other.

The aim of the attention layer is to provide the model with understanding the relevancy of the rest of the tokens to the token being processed.

In [165]:
outputs.encoder_attentions[1].shape

torch.Size([1, 8, 18, 18])


## Full Architecture
![animation.gif](https://heidloff.net/assets/img/2023/02/transformers.png)
[[source]](https://heidloff.net/article/foundation-models-transformers-bert-and-gpt/)

![animation.gif](https://jalammar.github.io/images/t/transformer_decoding_2.gif)
[[source]](https://jalammar.github.io/illustrated-transformer/)


## Tasks / Multiple objectives
As stated above Transformers can be used in multiple ways, depending on the desired output is. We can either use the whole architecture as shown above or use only the decoder or encoder. In every task the input is sequence of tokens. However the tasks differ depending on if they use the decoder or not. When using only the encoder t the output is either a label (text clf) a label for each token (token clf) or 2 indices which border the span of extracted answer. However when we use the decoder we then iteratively predict a single token (until a special token is generated, which symbolizes the end of the sentence.)

### Using Encoder only
The encoder hidden state provides contextual embeddings which can then be transformed to perform multiple tasks. The input is sequence where the output is either a label (text clf) a label for each token (token clf) or 2 indices which border the span of extracted answer
Heads:
* Text classification (positive/negative): The head is a linear layer processes the encoders final hidden states and transformes them into logits (which are then used as probabilities to classify the sentence)

* Token classification (NER): linear layer processes the hidden states to logits, where cross-entropy loss is computed, to return a most probable label for each token

* Extractive QA: linear layer processes to the start and end logits,  cross-entropy loss -> find best candidates for start and end of the answer in sequence

### Using Decoder only
* Text generation: The input sequence embeddings are parsed straight into the decoder, which then generates the most probable following tokens (the last hidden state of the decoder is processed by a linear layer and once more we get logits from which we generate the most probable token.

### Encoder-Decoder
* Sequence to sequence (Summarization, Translation): These tasks use the encoder and the decoder to process the input sequence. It is a text generation task (as discused above), however instead of continuing to generate based on an input prompt we also have the encoder to provide the contextual information of the input sequence, which we then then process. Both of these tasks are mainly about transcribing the input into a different text (translate to a different language / simplify the text but maintain the meaning).

## Pre-training & Fine-tuning
Pre-training and Fine-tuning

The magic of Deep language models lays in their good preliminary knowledge of language. They obtain this knowledge during so-called pre-training phase, where they are trained for a different token classification task an instance of Langauge Modeling.


### Language Modeling

In the instances of Language Modeling task, models are asked to solve a task of 'guessing' the **right word in the context**.

This task comes in two main instances:

1. **Masked Language Modeling (MLM)**: Models guess the correct token **within context**. This objective best prepares the model for **classification tasks** (Named Entity Recognition, Sequence Classification)

![image.png](https://www.rohanawhad.com/content/images/size/w1600/2022/04/image.png)
[[source]](https://www.rohanawhad.com/improvements-of-spanbert-over-bert/)

2. **Causal Langauge Modeling (CLM)**: Models guess the **following token** from previous context. This objective is better for preparing the model for **generation**, such as Dialogue, Summarization, Translation.

![image.png](https://gcdnb.pbrd.co/images/Bx4h6Lordx0y.png?o=1)  
![image.png](https://gcdnb.pbrd.co/images/rb7bmZS11gtl.png?o=1)
![image.png](https://gcdnb.pbrd.co/images/gXYffjzLIk7n.png?o=1)

Other variances: Synthetic token classification (ALBERT), Mask infilling (BART, T5), Sequence reconstruction (BART), Sequence Classification (BERT).


## Text Generation

Even though it may seem, that once we have a trained model we are done, there is the generation itself. While the weights in the decoder (and encoder) are set, we can still tweak the generated output, by using multiple generation strategies as well as using Logit Processors, which alter the logits from the linear layer and therefore alter the mostprobable next token. 

### What are Logits Processors?

In [166]:
from transformers import LogitsProcessor, LogitsProcessorList
import torch

class TestProcessor(LogitsProcessor):
    def __init__(self, max_length:int, eos_token_id: int):
        self.max_length = max_length
        self.eos_token_id = eos_token_id
        pass

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        if input_ids.shape[1] > self.max_length:
            scores[-1,:] = -float("inf")
            scores[-1, self.eos_token_id] = 0
        return scores

custom_processor = TestProcessor(5, tokenizer_t5.eos_token_id)

inputs = tokenizer_t5(input_sentence, return_tensors="pt")
outputs = model.generate(**inputs, logits_processor=LogitsProcessorList([custom_processor]))
output_text = tokenizer_t5.batch_decode(outputs, skip_special_tokens=True)

output_text

['Dies is the supercali']

### What can you create with Logits Processors?

A really good example of working with Logit processors is the jsonformers (https://github.com/1rgs/jsonformer) repo.

### Decoding strategies
While iteratively generating the output tokens, there are multiple strategies, that can be used to achieve the "best" output text.
* Greedy search: At each iteration, the most probable token is generated (most probable token at a time)


In [167]:
from transformers import AutoModelWithLMHead, AutoTokenizer
model_path = "gpt2"
input_sentence = "Two thousand years ago"
model_2 = AutoModelWithLMHead.from_pretrained(model_path)
tokenizer_2 = AutoTokenizer.from_pretrained(model_path)
input_ids = tokenizer_2(input_sentence, return_tensors="pt")
outputs=model_2.generate(**input_ids, early_stopping=True, max_length=50)
print(tokenizer_2.batch_decode(outputs, skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Two thousand years ago, the first humans were living in the world. Today, they are the only living species on Earth.\n\nThe first humans were living in the world. Today, they are the only living species on Earth. The first humans']


* Beam search: At each iteration $N$ number of beams with the best overall probability are stored. After the generation is complete the beam with highes overall probability is returned (most probable output as a whole)


In [168]:
outputs=model_2.generate(input_ids.input_ids, num_beams=2, num_return_sequences=2, max_length=50)
print(tokenizer_2.batch_decode(outputs, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


["Two thousand years ago, a group of scientists from the University of California, Berkeley, and the University of California, Santa Barbara, developed a method to measure the chemical composition of the Earth's crust. The team found that the Earth's crust is composed", "Two thousand years ago, a group of scientists from the University of California, Berkeley, and the University of California, Santa Barbara, developed a method to measure the chemical composition of the Earth's crust. The team found that the Earth's crust was composed"]


* Temperature: A parameter which adjusts the logits before applying the softmax function. Greater temperature can lead to less probable words generated (more unexpected generated sequence), small temperature will lead to more conservative outputs. 


In [169]:
outputs=model_2.generate(**input_ids, early_stopping=True, max_length=50, do_sample=True, top_k=0, temperature=0.7)
print(tokenizer_2.batch_decode(outputs, skip_special_tokens=True))
outputs=model_2.generate(**input_ids, early_stopping=True, max_length=50, do_sample=True, top_k=0, temperature = 0.001)
print(tokenizer_2.batch_decode(outputs, skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Two thousand years ago, the Arabs had been the most advanced people in the world. The slaves and the cattle of the Arabs had been domesticated. The Israelites were the first people in the world to produce a foodstuff that was not only dense']
['Two thousand years ago, the first humans were living in the world. Today, they are the only living species on Earth.\n\nThe first humans were living in the world. Today, they are the only living species on Earth. The first humans']


* Top-$K$ sampling: sampling only $K$ samples with the highest probabilities.  
* Nucleus sampling (top-$p$ sampling): Computing a cumulative distribution function and sampling only till the cut-off at the $p$ quantile


In [170]:
outputs=model_2.generate(input_ids.input_ids, do_sample=True, max_length=50, early_stopping=True)
print(tokenizer_2.batch_decode(outputs, skip_special_tokens=True))
outputs=model_2.generate(input_ids.input_ids, do_sample=True, top_k=50, max_length=50, early_stopping=True)
print(tokenizer_2.batch_decode(outputs, skip_special_tokens=True))
outputs=model_2.generate(input_ids.input_ids, do_sample=True, top_p=0.1, max_length=50, early_stopping=True)
print(tokenizer_2.batch_decode(outputs, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Two thousand years ago, for a while, you would have thought our history would lie far back in the Jurassic past.\n\nBut the dinosaurs were really out from under the sun, so a great deal had just been lost.\n\nOne story']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


["Two thousand years ago, the sun was formed and slowly, but slowly began to set in space, and we see it every time this sun sets (that's what we all assumed).\n\nOne of the most common misconceptions about the solar system is"]
['Two thousand years ago, the first humans were the first to colonize the planet. Today, the first humans are the first to colonize the planet.\n\nThe first humans were the first to colonize the planet. Today, the first humans']


# Hands-on: Adjusting the output without retraining

In this first short session we would ask you, to 

In [171]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, PhrasalConstraint
# model_path = "distilgpt2"
input_sentence =  "translate English to German: How old are you?"
input_sentence = "The soldiers \n \n"
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
constraints = [
    PhrasalConstraint(
        tokenizer("du", add_special_tokens=False).input_ids
    )
]
input_ids = tokenizer(input_sentence, return_tensors="pt")
outputs = model.generate(
    input_ids.input_ids,
    constraints=constraints,
    num_beams=10,
    num_return_sequences=1,
    no_repeat_ngram_size=1,
    remove_invalid_values=True,
)
# outputs=model.generate(input_ids.input_ids, do_sample=True, top_k=50, max_length=50, early_stopping=True)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['The soldiers were a army of the troops who had been killed in their wars and du']


In [172]:
tokenizer.convert_ids_to_tokens(tokenizer("hitler").input_ids)

['▁hit', 'ler', '</s>']